## Try 1


In [ ]:
import os
import pathlib
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image_dataset_from_directory
import glob
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
import splitfolders 

from tensorflow.keras.callbacks import ModelCheckpoint

In [49]:
image_path = pathlib.Path("../images")
DATA_DIR = pathlib.Path("../finetune_data")
train_path = DATA_DIR / "output/train"
val_path = DATA_DIR / "output/val"
test_path = DATA_DIR / "output/test"

In [42]:
Total_Images = glob.glob(f'{image_path}/*/*.jpg')
print("Total Number of Images", len(Total_Images))
Total_Images = pd.Series(Total_Images)

Total Number of Images 456


In [43]:
# sample visualization 

Total_Df = pd.DataFrame()

Total_Df['FileName'] = Total_Images.map(lambda ImageName :ImageName.split("H")[-1])

Total_Df['ClassId'] = Total_Images.map(lambda ImageName :ImageName.split("/")[-2])

Total_Df.head()

,FileName,ClassId
0,../images/extreme-close-up/casino.jpg,extreme-close-up
1,../images/extreme-close-up/psycho.jpg,extreme-close-up
2,../images/extreme-close-up/savingprivateryan.jpg,extreme-close-up
3,../images/extreme-close-up/capricorn-one.jpg,extreme-close-up
4,../images/extreme-close-up/goldfinger.jpg,extreme-close-up


In [44]:
# image counts 

Class_Id_Dist_Total = Total_Df['ClassId'].value_counts()
Class_Id_Dist_Total.head(10)

ClassId
close-up            145
medium-shot         140
long-shot           110
medium-long-shot     44
extreme-close-up     17
Name: count, dtype: int64

In [50]:
splitfolders.ratio(image_path, output=f"{DATA_DIR}/output", seed=101, ratio=(.8, .1, .1))

Copying files: 456 files [00:00, 5373.81 files/s]


In [51]:
train_path=f'{DATA_DIR}/output/train/'
val_path=f'{DATA_DIR}/output/val'
test_path=f'{DATA_DIR}/output/test'
class_names=os.listdir(train_path)
class_names_val=os.listdir(val_path)
class_names_test=os.listdir(test_path)

In [23]:
train_image1 = glob.glob(f'{DATA_DIR}/output/train/*/*.jpg')

Total_TrainImages = train_image1 
print("Total number of training images: ", len(Total_TrainImages))


test_image1 = glob.glob(f'{DATA_DIR}/output/test/*/*.jpg')

Total_TestImages = test_image1
print("Total number of test images: ", len(Total_TestImages))



Val_image1 = glob.glob(f'{DATA_DIR}/output/val/*/*.jpg')

Total_ValImages = Val_image1 
print("Total number of val images: ", len(Total_ValImages))

Total number of training images:  364
Total number of test images:  48
Total number of val images:  44


In [54]:
class_names = os.listdir(train_path)
print("Classes:", class_names)

# Load datasets
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

Classes: ['extreme-close-up', 'medium-shot', 'medium-long-shot', 'long-shot', 'close-up']


In [59]:
train_ds = image_dataset_from_directory(
    train_path,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="categorical"
)

val_ds = image_dataset_from_directory(
    val_path,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="categorical"
)

test_ds = image_dataset_from_directory(
    test_path,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="categorical"
)

Found 364 files belonging to 5 classes.
Found 44 files belonging to 5 classes.
Found 48 files belonging to 5 classes.


In [ ]:
## OPTIONAL: preprocessing: crop images to better represent aspect ratio

def crop_to_aspect(image, aspect_ratio=1.5):
    shape = tf.shape(image)
    h, w = shape[0], shape[1]
    target_w = tf.cast(h * aspect_ratio, tf.int32)
    target_h = tf.cast(w / aspect_ratio, tf.int32)

    # If too wide, crop width
    if w > target_w:
        offset_w = (w - target_w) // 2
        image = tf.image.crop_to_bounding_box(image, 0, offset_w, h, target_w)
    else:  # Else, crop height
        offset_h = (h - target_h) // 2
        image = tf.image.crop_to_bounding_box(image, offset_h, 0, target_h, w)
    return image

def preprocess(image, label):
    image = crop_to_aspect(image, aspect_ratio=1.5)
    image = tf.image.resize(image, [224, 224])  # Or 299, etc.
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

train_ds = train_ds.map(preprocess)
val_ds = val_ds.map(preprocess)
test_ds = test_ds.map(preprocess)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)

# Build model with ResNet50 base
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)
base_model.trainable = False  # Freeze base

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(class_names), activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
model.fit(train_ds, validation_data=val_ds, epochs=10)

Epoch 1/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 10s 633ms/step - accuracy: 0.3454 - loss: 2.5709 - val_accuracy: 0.5455 - val_loss: 1.3048
Epoch 2/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 560ms/step - accuracy: 0.6555 - loss: 1.1177 - val_accuracy: 0.6364 - val_loss: 1.0796
Epoch 3/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 553ms/step - accuracy: 0.7594 - loss: 0.7021 - val_accuracy: 0.5682 - val_loss: 1.0550
Epoch 4/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 554ms/step - accuracy: 0.7255 - loss: 0.7149 - val_accuracy: 0.5227 - val_loss: 1.2310
Epoch 5/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 552ms/step - accuracy: 0.8300 - loss: 0.4827 - val_accuracy: 0.6591 - val_loss: 1.0848
Epoch 6/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 552ms/step - accuracy: 0.8293 - loss: 0.4737 - val_accuracy: 0.5682 - val_loss: 1.2195
Epoch 7/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 556ms/step - accuracy: 0.8884 - loss: 0.3574 - val_accuracy: 0.5909 - val_loss: 1.2131
Epoch 8/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 553ms/step - accuracy: 0.9323 - loss: 0.2503 - val_accuracy: 0

In [73]:
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test accuracy: {test_acc:.2f}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 292ms/step - accuracy: 0.5278 - loss: 1.0595
Test accuracy: 0.54
